In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%%time
import pandas as pd
import pickle

pd.set_option('display.max_colwidth', -1)

PATH = 'Задача 1/input/'

df = pd.read_csv(PATH + 'train.csv', index_col='doc_id')
test_df = pd.read_csv(PATH + 'test.csv', index_col='doc_id')

CPU times: user 11.4 s, sys: 2.02 s, total: 13.5 s
Wall time: 13.7 s


## Sklearn Latent Dirichlet Allocation

### Обучение

Преобразуем данные в формат bag of words

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

features = 100000

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=100000,
                                stop_words=stopWords)

tf = tf_vectorizer.fit_transform(df['text'])

tf_feature_names = tf_vectorizer.get_feature_names()

CPU times: user 1min 24s, sys: 1.59 s, total: 1min 26s
Wall time: 1min 26s


50 тем оказалось вполне достаточным. 100 было слишком много, были очень похожие топики

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
n_components = 50
lda = LatentDirichletAllocation(n_components=n_components, max_iter=1,
                                learning_method='online', random_state=17,
                                verbose=True, n_jobs=-1)

In [0]:
lda.fit(tf)

In [0]:
pkl_filename = "/gdrive/My Drive/ITMO/pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lda, file)

In [0]:
def top_words(model, feature_names, n_top_words):
  l = []
  for topic in lda.components_:
    topic_tokens = " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
    l.append(topic_tokens)
  return pd.Series(data=l, name='topic_tokens')

In [0]:
topics_to_tokens = top_words(lda, tf_feature_names, 5)

In [112]:
topics_to_tokens[:10]

0    серия порядке сегодня сезон сериал     
1    любовь любить сердце люди знаешь       
2    людей люди хочется жизни умеешь        
3    жизнь жизни человека люди людей        
4    минут масло воды затем масла           
5    окружающим людей уважение людям следует
6    бизнес страница теги como сайт         
7    лишь свет руки любви детей             
8    женщина говорит мужчина муж сказала    
9    репост выбираем авы группу рандомно    
Name: topic_tokens, dtype: object

### Тестирование

In [49]:
%%time
test_tf = tf_vectorizer.transform(test_df['text'])
ans = lda.transform(test_tf)

CPU times: user 10.3 s, sys: 117 ms, total: 10.4 s
Wall time: 10.4 s


In [0]:
test_df['label'] = np.argmax(ans, axis=1)

Сохраним топик для каждого документа и популярные слова для каждого топика

In [0]:
test_df.to_csv('/gdrive/My Drive/ITMO/labels.csv', columns=['label'], index=True)
topics_to_tokens.to_csv('/gdrive/My Drive/ITMO/topics_tokens.csv', 
                        index_label='topic_id', header=True)

### Оценка

#### Количественная метрика

Для количественной оценки LDA можно воспользоваться [perplexity](http://qpleple.com/perplexity-to-evaluate-topic-models/). Perplexity - это показатель того, насколько хорошо вероятностная модель прогнозирует выборку

In [31]:
lda.perplexity(test_tf)

31362.778725074197

Данное знанчение довольно большое. Стоит попробовать разное количество топиков и посмотреть на их значение perplexity. В связи с этим получится подобрать оптимальное количество топиков

#### Качественная оценка (проверяя кластеры вручную)

In [86]:
test_df['label_tokens'] = test_df['label'].apply(lambda el: topics_to_tokens[el])
test_df.head(2)

,text,label,label_tokens
doc_id,,,
0,разнорабочий шеин примем работу разнорабочего универсальной работы складе строительных материвлов требования ответственность дисциплинированность трудолюбие обязанности выполнять погрузо разгрузочные работы совершать работу распаковке упаковке товара размещать товар складе условия график работы стабильные выплаты день предоставление рабочей формы отапливаемый склад оформление прямой работодатель писать звонить номеру работодатель шеин требуется расклейщик объявлений вопросы тел требуются рабочие установке счётчиков строительную фирму первый день научат установке сдельная оплата труда стоимость установки счётчика плановая норма выполнения работы счётчиков день рабочих работа городе липецке график работы дней липецке дня выходных привозят воронеж липецке снимают квартиру узнать подробнее телефону ищем сотрудника должность разнорабочий склад плата требования ответственность порядочность исполнительность опыт работы обязателен образование имеет значения обязанности погрузочно разгрузочные работы комплектация заказов размещение складе подготовка отгрузке доставки клиентам оформление необходимой документации отчетности актов приема товара товарно транспортных накладных внутренняя отчетность условия оформление работа центре города график присутствует корпоративное жильё сотрудников тел ооо армада преуспевающую компанию требуются энергичные целеустремленные активные сотрудники компания занимается арендой жилой недвижимости вакантное место менеджер работе клиентами молодой коллектив хорошие условия работы стабильный доход рублей месяц официальное трудоустройство согласно рабочие командировки города расположены филиалы компании хорошая бонусная система многое другое график работы возможностью плавающих выходных карьерный рост обучение тренинги получение нового опыта знаний сфере записи собеседование звонить тел срочно требуется магазин красное amp белое пеше стрелецкая специалист опп опыта работы руб график работы продавец опыта работы руб график работы контакты работа вахтовым методом воронеж требуются токаря обязанности токарная обработка металла станках требования опыт работы разряд условия вахта предоставление жилья оплата почасовая компенсация проезда оформление официальное дневная рабочая неделя часов контактное лицо наталья срочно оздоровительный центр требуются операторы телефон неполный рабочий день утренние вечерние смены офис центре города вопросы телефону кристина mybox срочно подработка требуются операторы телефон грффик работы сменный звонить номеру требуется продавец кассир магазин дома бристоль адресу шишкова зарплата белая рублей график дружный коллектив срочно звонить телефону,27,работы работа зарплата требуется работу
1,ၾကည ၾသဂ ၾကည ပစၥည ၾကည သတင unicode version သတင present meeting commander chief navy admiral tin aung san senior military officers office commander chief army commander north west command maj gen phone myat officers ranks families factory workers seikphyu station speech senior general said weapon modernization programmes underway transforming tatmadaw standard army knowledge promotion entire tatmadaw carried tatmadaw members capability handle advanced weapons tatmadaw members army navy air undergo training skilled using troop wise weapons food shelter needs tatmadaw members properly fulfi lled persons hour duty education promotion campaigns carried tatmadawmen wives outstanding students honoured duly medics assigned provide healthcare tatmadaw members families seriousness based service years promotions given tatmadaw members clear wrongdoings war veteran housings built retired military officers andother ranks allotted based budget funds war veteran housings built allotted tatmadaw owned agricultural livestock farms established respective units headquarters welfare tatmadawmen families fruits vegetables fish eggs meat produced farms distributed reduced prices tatmadawmen families need live discipline resolve difficulties requirements reporting respective officers accord

Как видно, второй пример был плохо "кластеризован". Но данный текст вообще на другом языке, возможно, CountVectorizer просто проигнорировал некоторые слова этого документа (из-за min_df=2), так как слов из данного документа нет в других документах.

In [103]:
s = set(tf_feature_names)
print(sum([word in s for word in test_df.loc[1, 'text'].split(' ')]), 
      'words of', len(test_df.loc[1, 'text'].split(' ')), 
      'in Count Vectorizer')

122 words of 315 in Count Vectorizer


Так и оказалось, почти треть слов не учитывается. Посмотрим на последние 2 документа

In [95]:
test_df.tail(2)

text  ...                                label_tokens
doc_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Здесь второй документ так же склассифицирован не очень хорошо. Проверим так же как и в прошлый раз

In [102]:
print(sum([word in s for word in test_df.loc[8099, 'text'].split(' ')]), 
      'words of', len(test_df.loc[8099, 'text'].split(' ')), 
      'in Count Vectorizer')

577 words of 650 in Count Vectorizer and


Но в данном тексте мног слов из label'а

In [109]:
label = set(test_df.loc[8099, 'label_tokens'].split())
sum([word in label for word in test_df.loc[8099, 'text'].split()])

8

Это довольно много, учитывая то, что не учитываются остальные слова, не вошедшие в топ 5

### Возможные улучшения

1) Попробовать другое количество кластеров, либо подобрать их в зависимости от perplexity [1]

2) Поменять *max_df* у *CountVectorizer* на 0.6 как написано [1]

3) Сделать лематизацию/стеминг, чтобы окончательно быть уверенным, что данные правильно подготовлены

4) Попробовать *TF-IDF* вместо *CountVectorizer*

[[1]](https://www.quora.com/What-are-good-ways-of-evaluating-the-topics-generated-by-running-LDA-on-a-corpus)